In [1]:
"""Volume 2: Non-negative Matrix Factorization."""

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
from imageio import imread
import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error as mse

In [37]:
#Problems 1-2
class NMFRecommender:

    def __init__(self,random_state=15,rank=3,maxiter=200,tol=1e-3):
        """The parameter values for the algorithm"""
        self.random_state = random_state
        self.rank = rank
        self.maxiter = maxiter
        self.tol = tol
    
    def initialize_matrices(self, m, n):
        """randomly initialize the W and H matrices,"""
        np.random.seed(self.random_state)
        W = np.random.random((m, self.rank))
        H = np.random.random((self.rank, n))
        return W, H
      
    def compute_loss(self, V, W, H):
        """Computes Frobenius norm of V - WH"""
        return np.linalg.norm(V - W@H, ord='fro')
    
    def update_matrices(self, V, W, H):
        """The multiplicative update step to update W and H"""
        H = np.multiply(H, np.divide(W.T @ V, W.T @ W @ H))
        W = np.multiply(W, np.divide(V @ H.T, W @ H @ H.T))
        return W, H
    
    def fit(self, V):
        """Fits W and H weight matrices using CVXPY"""
        iters = 0
        tol = 1e10
        self.W, self.H = self.initialize_matrices(V.shape[0], V.shape[1])
        while iters < self.maxiter or tol < self.tol:
            tol = self.compute_loss(V, self.W, self.H)
            iters += 1
            self.W, self.H = self.update_matrices(V, self.W, self.H)

    def reconstruct(self):
        """Reconstruct V matrix for comparison against the original V"""
        return self.W @ self.H

In [38]:
def prob4():
    """Run NMF recommender on the grocery store example"""
    V = np.array([[0, 1, 0, 1, 2, 2],
                  [2, 3, 1, 1 ,2, 2],
                  [1, 1, 1, 0, 1, 1],
                  [0, 2, 3, 4, 1, 1],
                  [0, 0, 0, 0, 1, 0]])
    nmf = NMFRecommender()
    nmf.fit(V)
    
    return nmf.W, nmf.H, sum(nmf.H[1] > nmf.H[0])
prob4()

(array([[2.09703201e+00, 3.39926395e-02, 1.24321860e-03],
        [1.17434435e+00, 1.86675479e-01, 1.75606037e+00],
        [4.31904447e-01, 3.45814238e-02, 8.86373271e-01],
        [2.60722324e-01, 2.04682328e+00, 1.78289205e-02],
        [4.53836297e-01, 7.67955446e-75, 1.59314155e-36]]),
 array([[2.00066951e-03, 4.50872847e-01, 3.74457967e-12, 4.34272058e-01,
         9.99489928e-01, 9.00805812e-01],
        [1.39455877e-18, 9.13590620e-01, 1.45389732e+00, 1.90276113e+00,
         3.54274515e-01, 3.68156008e-01],
        [1.13539583e+00, 1.22102968e+00, 5.48251136e-01, 3.46267683e-03,
         4.72274477e-01, 5.33422597e-01]]),
 3)

In [3]:
def prob5():
    """
    Calculate the rank and run NMF
    """
    df = pd.read_csv('artist_user.csv').set_index("Unnamed: 0")
    
    # Compute benchmark
    benchmark = np.linalg.norm(df, ord='fro') * .0001

    r = 3
    model = NMF(n_components = r)
    W = model.fit_transform(df)
    H = model.components_
    error = np.sqrt(mse(df, W @ H))
    
    # Iteratively find best rank
    while True:
        if error < benchmark:
            break
        r += 1
        model = NMF(n_components = r)
        W = model.fit_transform(df)
        H = model.components_
        error = np.sqrt(mse(df, W @ H))
    return r, W @ H

In [23]:
def discover_weekly(id):
    """
    Create the recommended weekly 30 list for a given user
    """
    df = pd.read_csv('artist_user.csv').set_index("Unnamed: 0")
    _, V = prob5()
    
    # Get the index of that user to get the correct row in V
    ind = list(df.index.values).index(id)
    listened = df.loc[id]
    scores = V[ind]
    
    # Find which songs are recommended to them the most
    argsorted = list(np.argsort(-scores))
    artists = pd.read_csv('artists.csv')
    
    recs = []
    i = 0
    # Go through the top recommendations, leaving out the songs they've
    # already listened to, add the ones that they haven't heard
    while len(recs) < 30:
        artist_ind = argsorted.index(i)
        if listened.iloc[artist_ind] == 0:
            recs.append(artists.iloc[artist_ind]['name'])
        i += 1
    return recs

discover_weekly(2)

['Fabrizio De André',
 'Antonio Vivaldi',
 'Thavius Beck',
 'Mano Solo',
 'Dutch',
 'Six Magics',
 'Worm Is Green',
 'J Dilla',
 'Kate Miller-Heidke',
 'Stars',
 'The Contortionist',
 'Edge of Dawn',
 'Frank Zappa',
 'Kim Carnes',
 'Afrika Bambaataa',
 'Angerfist',
 'Uncle Murda',
 'Emma Bunton',
 'Ivan$Stani',
 'Metal Church',
 'Prototypes',
 'Pretty Lights',
 'Sansar',
 "Guns N' Roses",
 'Norther',
 'Emma Shapplin',
 'Radiotape',
 'Planet Funk',
 'IOSYS',
 'yelworC']

,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,...,20836,20837,20838,20839,20840,20841,20842,20843,20844,20845
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
